<a href="https://colab.research.google.com/github/tomonari-masada/course2024-sml/blob/main/13_2d_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2次元の可視化

* 2次元の可視化は、もとは高次元のデータセットの次元を、2次元へと圧縮することである。
  * 前回の演習をふまえると・・・
  * 2次元のような非常に低次元の空間への次元削減は・・・
  * それほど自明なデータ処理ではなさそうだが・・・

## 準備

* UMAPだけ、scikit-learnに実装されていない。

In [ ]:
!pip install umap-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_distances
import seaborn as sns
from umap import UMAP

%config InlineBackend.figure_format = "retina"

## データセット
* 今回はdigits dataを題材として使う。

In [ ]:
digits = datasets.load_digits()

In [ ]:
digits.data.shape

In [ ]:
digits.target.shape

In [ ]:
plt.imshow(digits.data[0].reshape(8, -1), cmap="gray");

In [ ]:
digits.target[:20]

## あえてデータセットにnoisyなインスタンスを追加する

In [ ]:
(digits.target == 0).sum()

In [ ]:
digits.data[0]

In [ ]:
np.random.randint(0, high=15, size=64)

In [ ]:
plt.imshow(np.random.randint(0, high=15, size=64).reshape(8, -1), cmap="gray");

In [ ]:
n_noisy_images = 178
noisy_images = np.random.randint(0, high=15, size=(n_noisy_images, 64))
noisy_images.shape

In [ ]:
np.full(n_noisy_images, 10)

In [ ]:
np.concatenate([digits.data, noisy_images]).shape

In [ ]:
np.concatenate([digits.target, np.full(n_noisy_images, 10)]).shape

In [ ]:
digits.data = np.concatenate([digits.data, noisy_images])
digits.target = np.concatenate([digits.target, np.full(n_noisy_images, 10)])

## 可視化のためのヘルパ関数
* もっと凝った可視化をしたいなら、下記を参照。
 * https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py

In [ ]:
def scatter_plot(embedding, target, cmap=plt.get_cmap("tab20"), ax=None):
  for color in np.unique(target):
    indices = (target == color)
    if ax is None:
      plt.scatter(embedding[indices, 0], embedding[indices, 1], label=color, color=cmap(color), s=3, alpha=0.5)
    else:
      ax.scatter(embedding[indices, 0], embedding[indices, 1], label=color, color=cmap(color), s=3, alpha=0.5)

## PCAによる可視化

In [ ]:
pca = PCA(10)
embedding = pca.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend()
plt.title("digits data embedded into two dimensions by PCA");

## UMAPによる可視化
* https://umap-learn.readthedocs.io/en/latest/parameters.html


* どんな可視化ツールにも、調整できるパラメータがある。
* UMAPの場合は・・・
  * パラメータ`n_neighbors`を変えると可視化がどう変わるか。
  * パラメータ`min_dist`を変えると可視化がどう変わるか。


### デフォルトの設定で可視化

In [ ]:
%%time
reducer = UMAP(random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend()
plt.title("digits data embedded into two dimensions by UMAP");

* ノイズ画像なしだと？

In [ ]:
digits.data[:-n_noisy_images,:].shape

In [ ]:
digits.target[:-n_noisy_images].shape

In [ ]:
%%time
reducer = UMAP(random_state=42)
embedding = reducer.fit_transform(digits.data[:-n_noisy_images,:])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target[:-n_noisy_images])
plt.setp(ax, xticks=[], yticks=[])
plt.legend()
plt.title("digits data embedded into two dimensions by UMAP");

### `n_neighbors`を変更する
* デフォルトの値は15

In [ ]:
def draw_umap(n_neighbors=15, min_dist=0.1, title=""):
  reducer = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, random_state=42)
  u = reducer.fit_transform(digits.data)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  scatter_plot(u, digits.target, ax=ax)
  plt.setp(ax, xticks=[], yticks=[])
  #plt.legend()
  plt.title(title, fontsize=15);

In [ ]:
for n in (5, 10, 20, 50, 100, 200):
  title = f"n_neighbors = {n}"
  print(title)
  draw_umap(n_neighbors=n, title=title)

### `min_dist`を変更する
* デフォルトの値は0.1

In [ ]:
for d in (0.0, 0.1, 0.25, 0.5, 0.8, 0.99):
  title = f"min_dist = {d}"
  print(title)
  draw_umap(min_dist=d, title=title)

## t-SNEによる可視化
* https://scikit-learn.org/stable/auto_examples/manifold/plot_t_sne_perplexity.html
 * パラメータ`perplexity`を変えると可視化がどう変わるか。

### デフォルトの設定で可視化

In [ ]:
%%time
reducer = manifold.TSNE(random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend()
plt.title("digits data embedded into two dimensions by t-SNE");

### `perplexity`を変更する

In [ ]:
def draw_tsne(perplexity=30.0, title=""):
  reducer = manifold.TSNE(perplexity=perplexity, random_state=42)
  u = reducer.fit_transform(digits.data)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  scatter_plot(u, digits.target, ax=ax)
  plt.setp(ax, xticks=[], yticks=[])
  plt.legend()
  plt.title(title, fontsize=15);

In [ ]:
for p in (2, 5, 10, 20, 50, 100):
  title = f"perplexity = {p}"
  print(title)
  draw_tsne(perplexity=p, title=title)

## MDS

### デフォルトの設定で可視化

In [ ]:
%%time
reducer = manifold.MDS(normalized_stress=False, random_state=42)
embedding = reducer.fit_transform(digits.data)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
scatter_plot(embedding, digits.target)
plt.setp(ax, xticks=[], yticks=[])
plt.legend()
plt.title("digits data embedded into two dimensions by MDS");

* epsを変更してみたが、今回のデータセットでは、ほとんど変化が見られなかった。

## 考察
* それぞれの可視化ツールを、デフォルトの設定で使っても構わないか？
* 2次元の可視化において、遠いものは遠いと言っていいか？
* 2次元の可視化において、近いものは近いと言っていいか？
* 例えば、digits dataに関して、ここまでの可視化を踏まえて、結論して構わないことは、何か？


## 助言
* 初めに結論ありきの、"自分が見たいものだけを見る可視化"にならないよう、注意しよう。
* 最低限、複数の可視化手法を比較し、共通して見られる特徴は何かを確認しよう。